#### Description

The code is used for managing Gmail data. 


#### Configuration

1. Enable Gmail SDK in Google Developer Console.
3. Setup API keys, OAuth Client ID and the scopes needed, Service account.
4. Download OAuth credential json file from Google Developer Console. The file likes "client_secret_274636580459-mlfmiu7frnf84hm794shl21tdmal10ee.apps.googleusercontent.com.json".
5. Rename the file as "credentials.json" and put it in the root directory.

#### Reference

- Gmal API
https://developers.google.com/gmail/api/quickstart/python

- OAuth 2.0 Scopes
https://developers.google.com/identity/protocols/oauth2/scopes

- People API
https://developers.google.com/people/quickstart/python


In [13]:
from __future__ import print_function

import os.path
import pandas as pd
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [29]:
tmp_user_obj, tmp_user_list = None, None
tmp_message_obj1, tmp_message_obj2, tmp_message_list = None, None, None

tmp_attachment_obj1, tmp_attachment_obj2, tmp_attachment_list = None, None, None

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    # Call the Gmail API
    service = build('gmail', 'v1', credentials=creds)
    
    tmp_user_obj = service.users()
    tmp_user_profile = service.users().getProfile(userId='me').execute()
    
    tmp_message_obj1 = service.users().messages()
    tmp_message_list = service.users().messages().list(userId='me').execute()
    
    tmp_attachment_obj1 = service.users().messages().attachments()
#     tmp_attachment_list = service.users().messages().attachments().get(userId='me').execute()

    # label
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])
    if not labels:
        print('No labels found.')
    print('Labels:')
    for label in labels:
        print(label['name'], end=' ')
    print()
    
    # email List
    response_list = service.users().messages().list(userId='me').execute()
    print(response_list['messages'][:5])
    print()
    
    # An email Content
    message = service.users().messages().get(userId='me', id=response_list['messages'][0]['id']).execute()
    print(base64.urlsafe_b64decode(message['payload']['parts'][0]['body']['data'].encode('ascii')).decode("utf-8"))
    
    

except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f'An error occurred: {error}')

Labels:
CHAT SENT INBOX IMPORTANT TRASH DRAFT SPAM CATEGORY_FORUMS CATEGORY_UPDATES CATEGORY_PERSONAL CATEGORY_PROMOTIONS CATEGORY_SOCIAL STARRED UNREAD 


minimal - Returns only email message ID and labels; does not return the email headers, body, or payload.

full - Returns the full email message data with body content parsed in the payload field; the raw field is not used. Format cannot be used when accessing the api using the gmail.metadata scope.

raw	- Returns the full email message data with body content in the raw field as a base64url encoded string; the payload field is not used. Format cannot be used when accessing the api using the gmail.metadata scope.

metadata - Returns only email message ID, labels, and email headers.